In [1]:
# script to load dimension

In [2]:
# Import required libraries
import sys
from lib.spark_session import get_spark_session
from lib.utils import date_data, get_string_cols, get_rundate
from lib.job_control import insert_log, get_max_timestamp
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import current_timestamp, expr, to_date, date_format, udf
from pyspark.sql.types import StringType
from datetime import datetime
from delta import DeltaTable
import uuid

In [3]:
# JOB Parameters
rundate = get_rundate()
schema_name = "edw"
table_name = "dim_store"
table_full_name = f"{schema_name}.{table_name}"
staging_table_full_name = "edw_stg.dim_store_stg"
print("SPARK_APP: JOB triggered for rundate - " + rundate)

SPARK_APP: JOB triggered for rundate - 20220101


In [4]:
# Generate Spark Session
spark: SparkSession = get_spark_session(f"Dimension load - {table_full_name}")
print("SPARK_APP: Spark UI - " + spark.sparkContext.uiWebUrl)

SPARK_APP: Spark UI - http://16804892cba9:4040


In [5]:
# Spark Configs
spark.conf.set("spark.sql.shuffle.partitions", 8)

In [6]:
# Read data from Staging
df_stg = spark \
    .read \
    .table(staging_table_full_name)

print("SPARK_APP: Staging Data Count - " + str(df_stg.count()))
print("SPARK_APP: Printing Staging Schema --")
df_stg.printSchema()

SPARK_APP: Staging Data Count - 7
SPARK_APP: Printing Staging Schema --
root
 |-- store_id: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- insert_dt: timestamp (nullable = true)
 |-- rundate: string (nullable = true)
 |-- update_dt: timestamp (nullable = true)



In [7]:
# Generated uuid UDF for Surrogate Key
uuidUDF = udf(lambda : str(uuid.uuid4()),StringType())

In [8]:
# Generate SURROGATE KEYs
df_dim_temp = df_stg.withColumn("row_wid", uuidUDF())

print("SPARK_APP: Dim Temp Data Count - " + str(df_dim_temp.count()))
print("SPARK_APP: Printing Dim Temp Schema --")
df_dim_temp.printSchema()

SPARK_APP: Dim Temp Data Count - 7
SPARK_APP: Printing Dim Temp Schema --
root
 |-- store_id: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- insert_dt: timestamp (nullable = true)
 |-- rundate: string (nullable = true)
 |-- update_dt: timestamp (nullable = true)
 |-- row_wid: string (nullable = true)



In [11]:
# Get the delta table for Upserts (SCD1)
dt_dim = DeltaTable.forName(spark, table_full_name)

#
dt_dim.toDF().printSchema()

# Validate if the load is full load
if get_max_timestamp(spark, schema_name, table_name) == "1900-01-01 00:00:00.000000":
    print("SPARK_APP: Table is set for full load") 
    # Truncate the Dimension table
    spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)
    dt_dim.delete(f"1=1")
    dt_dim.vacuum(0)
    
# Create the UPSERT logic
dt_dim.alias("dim_store") \
    .merge(df_dim_temp.alias("dim_temp"), "dim_store.store_id = dim_temp.store_id") \
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll() \
    .execute()

root
 |-- row_wid: string (nullable = true)
 |-- store_id: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- rundate: string (nullable = true)
 |-- insert_dt: timestamp (nullable = true)
 |-- update_dt: timestamp (nullable = true)

SPARK_APP: Table is set for full load


In [12]:
# Add job details in JOB CONTROL
insert_log(spark, schema_name, table_name, datetime.now(), rundate)
print("SPARK_APP: Update JOB Control Log")

SPARK_APP: Update JOB Control Log


In [13]:
spark.sql(f"select * from edw.job_control where table_name = '{table_name}' order by insert_dt desc limit 1").show(truncate=False)

+-----------+----------+--------------------------+--------+--------------------------+
|schema_name|table_name|max_timestamp             |rundate |insert_dt                 |
+-----------+----------+--------------------------+--------+--------------------------+
|edw        |dim_store |2025-12-29 22:03:06.875007|20220101|2025-12-29 22:03:07.502236|
+-----------+----------+--------------------------+--------+--------------------------+



In [14]:
# Get the logs from delta table version
dt_dim.history().limit(1).select("version","operationMetrics.executionTimeMs", 
                                 "operationMetrics.numTargetRowsInserted",
                                "operationMetrics.numTargetRowsUpdated",
                                "operationMetrics.numOutputRows").show(1, False)

+-------+---------------+---------------------+--------------------+-------------+
|version|executionTimeMs|numTargetRowsInserted|numTargetRowsUpdated|numOutputRows|
+-------+---------------+---------------------+--------------------+-------------+
|1      |7977           |7                    |0                   |7            |
+-------+---------------+---------------------+--------------------+-------------+



In [15]:
# Generate Symlink manifest for Athena Access
dt = DeltaTable.forName(spark, table_full_name)
dt.generate("symlink_format_manifest")
print("SPARK_APP: Symlink Manifest file generated")

SPARK_APP: Symlink Manifest file generated


In [16]:
spark.sql("select * from edw.dim_store").show()

+--------------------+--------+--------------+-----------+-----+--------+--------------+--------+--------------------+--------------------+
|             row_wid|store_id|       address|       city|state|zip_code|  phone_number| rundate|           insert_dt|           update_dt|
+--------------------+--------+--------------+-----------+-----+--------+--------------+--------+--------------------+--------------------+
|301d5bcd-e20e-431...|    S002|    456 Elm St|Anothertown|   MH|   67890|91-99999-99999|20220101|2025-12-28 18:08:...|2025-12-28 18:08:...|
|389e771e-fbac-48c...|    S004|321 Birch Blvd| Small Town|   OR|   76684|91-88822-00000|20220101|2025-12-28 18:08:...|2025-12-28 18:08:...|
|f220bd3e-7198-401...|    S005|   654 Pine St|   Busytown|   WB|   11111|91-00002-22222|20220101|2025-12-28 18:08:...|2025-12-28 18:08:...|
|f97ec2b2-17b6-4e6...|    S006|  987 Cedar Rd|  Hill Town|   JK|   22222|91-33330-33333|20220101|2025-12-28 18:08:...|2025-12-28 18:08:...|
|3adfc3b8-cac7-4ae..

In [18]:
spark.stop()